In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
from premise import *
import brightway2 as bw
bw.projects.set_current("premise")

In [62]:
ndb = NewDatabase(
    scenarios = [
        {"model":"image", "pathway":"SSP2-RCP26", "year":2055,},
        #{"model":"image", "pathway":"SSP2-RCP26", "year":2040,},
        #{"model":"image", "pathway":"SSP2-RCP19", "year":2040,},
    ],        
    source_db="ecoinvent 3.8 cutoff",
    source_version="3.8",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
    system_model="consequential"
)

+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+

////////////////////// EXTRACTING SOURCE DATABASE //////////////////
Done!

//////////////////// IMPOR

In [ ]:
di = {(True, True, True, False): {"year":10, "avg": 5}}

In [13]:
a = 2
b = 3
c = 4
d = None
di[(bool(a), bool(b), bool(c), bool(d))]

{'year': 10, 'avg': 5}